In [ ]:
# Abstract-text-cleaning:leave numbers!
# remove special characters and stop words (delete word indicate tense; creat a new stopword list)
#(note: tense can help with the classification; protocol : future tense ; compeleted RCT: past tense)
#tokenlization and lemma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import pickle
import spacy
import numpy as np
from spacy.lang.en import English

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
nlp = spacy.load('en')

In [ ]:
nlp.Defaults.stop_words -= {"will","had","was","were","did"}

In [ ]:
new_stopwords = spacy_stopwords

In [ ]:
def preprocess_text(df):
    df.AbstractPlusTitle = df.AbstractPlusTitle.fillna(' ')
    df.AbstractPlusTitle = df.AbstractPlusTitle.str.replace('\n',' ')
    df.AbstractPlusTitle = df.AbstractPlusTitle.str.replace('\r',' ')
    return df

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/AugmDataset_addTil0.pkl')
#df_comb = df_comb.reset_index(drop =  True)

In [ ]:
df = df.reset_index()

In [ ]:
df.head()

,index,Title,JournalInfo,Abstract,TiL,AbL,AbstractPlusTitle
0,0,Qigong or Tai Chi in Cancer Care: an Updated S...,Curr Oncol Rep,Qigong and Tai Chi are two increasingly popula...,0,0,Qigong and Tai Chi are two increasingly popula...
1,2,Tai Chi and Qigong for cancer-related symptoms...,J Cancer Surviv,This study aims to summarize and critically ev...,0,0,This study aims to summarize and critically ev...
2,3,The effectiveness of tai chi in breast cancer ...,Complement Ther Clin Pract,Tai chi has been suggested as a potential effe...,0,0,Tai chi has been suggested as a potential effe...
3,6,Effectiveness of Tai Chi on fibromyalgia patie...,Complement Ther Med,To identify empirical evidence on the effectiv...,0,0,To identify empirical evidence on the effectiv...
4,7,Does Tai Chi Chuan improve psychological well-...,Medicine Baltimore,Breast cancer is the most prevalent cancer in ...,0,0,Breast cancer is the most prevalent cancer in ...


In [ ]:
df = df[['AbstractPlusTitle','AbL']]

In [ ]:
preprocess_text(df)

,AbstractPlusTitle,AbL
0,Qigong and Tai Chi are two increasingly popula...,0
1,This study aims to summarize and critically ev...,0
2,Tai chi has been suggested as a potential effe...,0
3,To identify empirical evidence on the effectiv...,0
4,Breast cancer is the most prevalent cancer in ...,0
...,...,...
542,Fibromyalgia is a condition characterised by c...,1
543,Mild cognitive impairment (MCI) is cognitive d...,0
544,There is no sufficient evidence on the effecti...,0
545,Many of the currently available treatments for...,0


In [ ]:
def clean_text(x):
  #x=re.sub(r"\'ll"," will ",x) ----  In academic paper, people tend not to use contraction
  #x=re.sub('[^a-zA-Z0-9]', ' ',x).lower()
  x = re.sub('[^a-zA-Z]', ' ',x).lower()
  x = x.split()
  new_list = []
  for word in x:
    if word not in new_stopwords: 
      new_list.append(word)
  return " ".join(new_list)

In [ ]:
df['AbstractPlusTitle'] = df.AbstractPlusTitle.apply(lambda x:clean_text(x))

In [ ]:
#df_comb[df_comb.index.duplicated(keep = False)]

In [ ]:
X = df.loc[:,'AbstractPlusTitle']

In [ ]:
X

0      qigong tai chi increasingly popular mind body ...
1      study aims summarize critically evaluate effec...
2      tai chi suggested potential effective interven...
3      identify empirical evidence effectiveness tai ...
4      breast cancer prevalent cancer women worldwide...
                             ...                        
542    fibromyalgia condition characterised chronic w...
543    mild cognitive impairment mci cognitive declin...
544    sufficient evidence effectiveness acupuncture ...
545    currently available treatments obsessive compu...
546    chronic fatigue cf unexplained fatigue lasting...
Name: AbstractPlusTitle, Length: 547, dtype: object

In [ ]:
y = df.loc[:,'AbL']

In [ ]:
y

0      0
1      0
2      0
3      0
4      0
      ..
542    1
543    0
544    0
545    0
546    1
Name: AbL, Length: 547, dtype: int64

In [ ]:
df.to_pickle('/content/drive/MyDrive/preprodf_Augm.pkl')

In [ ]:
#df['Abstract'] = df.Abstract.apply(lambda x: x.split())

In [ ]:
#df['Length'] = df.Abstract.apply(lambda x: len(x))

In [ ]:
#df.describe()

# DistillBERTs

In [ ]:
!pip3 install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)


In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 124kB/s 
     |████████████████████████████████| 6.8MB 50.8MB/s 
     |████████████████████████████████| 983kB 46.9MB/s 
     |████████████████████████████████| 266kB 52.5MB/s 
     |████████████████████████████████| 1.9MB 47.9MB/s 
     |████████████████████████████████| 1.2MB 48.7MB/s 
     |████████████████████████████████| 471kB 35.1MB/s 
     |████████████████████████████████| 3.3MB 48.7MB/s 
     |████████████████████████████████| 901kB 59.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=20736a8c370aaeeef91368a3e942d1da6eb65a4d827476e3f4fdd715e20e1567
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=2244acb7ecfb70b72cfb60c48e7ea8625e2c56a9972819ee2fd2fb563ddbb243
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/preprodf_Augm.pkl')

In [ ]:
df_valid = df.sample(frac = 0.2, random_state = 42)

In [ ]:
type(df_valid)

pandas.core.frame.DataFrame

In [ ]:
#df_valid.to_pickle('/content/drive/MyDrive/AbsTil_valid.pkl')

In [ ]:
df_train = df.drop(df_valid.index)

In [ ]:
df_train

,AbstractPlusTitle,AbL
1,study aims summarize critically evaluate effec...,0
3,identify empirical evidence effectiveness tai ...,0
4,breast cancer prevalent cancer women worldwide...,0
5,post stroke mental disorders psmds post stroke...,0
7,insomnia prevalent significant public health c...,0
...,...,...
541,study participants female fibromyalgia fm were...,0
543,mild cognitive impairment mci cognitive declin...,0
544,sufficient evidence effectiveness acupuncture ...,0
545,currently available treatments obsessive compu...,0


In [ ]:
type(df_train)

pandas.core.frame.DataFrame

In [ ]:
#df_train.to_pickle('/content/drive/MyDrive/AbsTil_train.pkl')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
#df_train = pd.read_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/BOW_Distill_BertS/AbsTil_train.pkl')
#df_valid = pd.read_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/BOW_Distill_BertS/AbsTil_valid.pkl')

In [ ]:
import tensorflow as tf
from sklearn import metrics
import ktrain
from ktrain import text

In [ ]:
train_sizes = [10,50,100,200,300]
for size in train_sizes:
    #print(type(size))
     df_train = df.drop(df_valid.index)
     df_train = df_train.sample(n=size)
     print(type(df_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
import ktrain
from ktrain import text
# df = pd.read_pickle('/content/drive/MyDrive/preprodf_Augm.pkl')
# df_valid = df.sample(frac = 0.2, random_state = 42)
train_sizes = [10,50,100,200,300]
df_train = df.drop(df_valid.index)
for size in train_sizes:
#   df_train = df.drop(df_valid.index)
    df_tra = df_train.sample(n=size)
    (train, val, preproc) = text.texts_from_df(train_df=df_tra, 
                                               text_column='AbstractPlusTitle', 
                                               label_columns='AbL',
                                               val_df = df_valid,
                                               maxlen = 512, #length of each doc aligned to 512
                                               preprocess_mode = 'distilbert')
    bertmodel = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)
    learner = ktrain.get_learner(model = bertmodel,
                             train_data = train,
                             val_data = val,
                             batch_size = 5)
    learner.fit_onecycle(lr = 2e-5, epochs=10)
    predictor = ktrain.get_predictor(learner.model, preproc)
    textval=df_valid['AbstractPlusTitle']
    data=list(textval)
    y_pred = predictor.predict(data, return_proba=True)
    df_valid.AbL = df_valid.AbL.astype('int')
    df_tra.AbL = df_tra.AbL.astype('int')
    y_test = df_valid['AbL']
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:,1])
    auc = metrics.auc(fpr, tpr)
    print(auc)

['not_AbL', 'AbL']
     not_AbL  AbL
230      0.0  1.0
125      1.0  0.0
541      1.0  0.0
284      0.0  1.0
296      0.0  1.0
['not_AbL', 'AbL']
     not_AbL  AbL
176      1.0  0.0
77       1.0  0.0
361      1.0  0.0
90       1.0  0.0
368      1.0  0.0
preprocessing train...
language: en
train sequence lengths:
	mean : 162
	95percentile : 203
	99percentile : 205


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
2/2 [==============================] - 11s 4s/step - loss: 0.6766 - accuracy: 0.6000 - val_loss: 0.6829 - val_accuracy: 0.6055
Epoch 2/10
2/2 [==============================] - 3s 3s/step - loss: 0.6988 - accuracy: 0.3333 - val_loss: 0.6820 - val_accuracy: 0.6055
Epoch 3/10
2/2 [==============================] - 3s 3s/step - loss: 0.6898 - accuracy: 0.4667 - val_loss: 0.6820 - val_accuracy: 0.6055
Epoch 4/10
2/2 [==============================] - 3s 3s/step - loss: 0.7095 - accuracy: 0.4000 - val_loss: 0.6816 - val_accuracy: 0.6055
Epoch 5/10
2/2 [==============================] - 3s 3s/step - loss: 0.6706 - accuracy: 0.4667 - val_loss: 0.6826 - val_accuracy: 0.6055
Epoch 6/10
2/2 [==============================] - 3s 3s/step - loss: 0.6699 - accuracy: 0.8000 - val_loss: 0.6836 - val_accuracy: 0.6422
Epoch 7/10
2/2 [==============================] - 3s 3s/step - loss: 0.6

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
10/10 [==============================] - 14s 718ms/step - loss: 0.6844 - accuracy: 0.6791 - val_loss: 0.6803 - val_accuracy: 0.6055
Epoch 2/10
10/10 [==============================] - 5s 563ms/step - loss: 0.6890 - accuracy: 0.5420 - val_loss: 0.6771 - val_accuracy: 0.6055
Epoch 3/10
10/10 [==============================] - 5s 571ms/step - loss: 0.6752 - accuracy: 0.6122 - val_loss: 0.6687 - val_accuracy: 0.6055
Epoch 4/10
10/10 [==============================] - 6s 582ms/step - loss: 0.6846 - accuracy: 0.5079 - val_loss: 0.6618 - val_accuracy: 0.6055
Epoch 5/10
10/10 [==============================] - 6s 587ms/step - loss: 0.6551 - accuracy: 0.5344 - val_loss: 0.6508 - val_accuracy: 0.6055
Epoch 6/10
10/10 [==============================] - 6s 587ms/step - loss: 0.6126 - accuracy: 0.7151 - val_loss: 0.5991 - val_accuracy: 0.6422
Epoch 7/10
10/10 [========================

...

Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
20/20 [==============================] - 18s 501ms/step - loss: 0.6951 - accuracy: 0.4917 - val_loss: 0.6928 - val_accuracy: 0.5321
Epoch 2/10
20/20 [==============================] - 9s 436ms/step - loss: 0.6906 - accuracy: 0.4989 - val_loss: 0.6831 - val_accuracy: 0.7615
Epoch 3/10
20/20 [==============================] - 9s 445ms/step - loss: 0.6803 - accuracy: 0.5890 - val_loss: 0.6756 - val_accuracy: 0.5596
Epoch 4/10
20/20 [==============================] - 9s 456ms/step - loss: 0.6697 - accuracy: 0.6491 - val_loss: 0.5780 - val_accuracy: 0.8073
Epoch 5/10
20/20 [==============================] - 9s 450ms/step - loss: 0.5092 - accuracy: 0.8635 - val_loss: 0.4810 - val_accuracy: 0.8165
Epoch 6/10
20/20 [==============================] - 9s 440ms/step - loss: 0.2862 - accuracy: 0.9436 - val_loss: 0.4266 - val_accuracy: 0.8349
Epoch 7/10
20/20 [========================

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
40/40 [==============================] - 24s 409ms/step - loss: 0.6853 - accuracy: 0.5393 - val_loss: 0.6669 - val_accuracy: 0.6055
Epoch 2/10
40/40 [==============================] - 15s 388ms/step - loss: 0.6750 - accuracy: 0.5612 - val_loss: 0.6553 - val_accuracy: 0.6055
Epoch 3/10
40/40 [==============================] - 15s 382ms/step - loss: 0.6604 - accuracy: 0.5805 - val_loss: 0.6124 - val_accuracy: 0.8073
Epoch 4/10
40/40 [==============================] - 15s 371ms/step - loss: 0.6313 - accuracy: 0.6671 - val_loss: 0.6778 - val_accuracy: 0.5505
Epoch 5/10
40/40 [==============================] - 15s 369ms/step - loss: 0.5731 - accuracy: 0.6799 - val_loss: 0.3836 - val_accuracy: 0.8349
Epoch 6/10
40/40 [==============================] - 15s 373ms/step - loss: 0.4819 - accuracy: 0.7608 - val_loss: 0.3782 - val_accuracy: 0.8257
Epoch 7/10
40/40 [===================

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
60/60 [==============================] - 30s 377ms/step - loss: 0.6880 - accuracy: 0.5549 - val_loss: 0.6587 - val_accuracy: 0.6055
Epoch 2/10
60/60 [==============================] - 22s 364ms/step - loss: 0.6618 - accuracy: 0.5878 - val_loss: 0.5667 - val_accuracy: 0.6514
Epoch 3/10
60/60 [==============================] - 21s 353ms/step - loss: 0.5379 - accuracy: 0.7196 - val_loss: 0.4814 - val_accuracy: 0.7798
Epoch 4/10
60/60 [==============================] - 21s 350ms/step - loss: 0.4013 - accuracy: 0.8531 - val_loss: 0.3540 - val_accuracy: 0.8532
Epoch 5/10
60/60 [==============================] - 21s 354ms/step - loss: 0.2400 - accuracy: 0.9279 - val_loss: 0.3589 - val_accuracy: 0.8716
Epoch 6/10
60/60 [==============================] - 21s 355ms/step - loss: 0.2006 - accuracy: 0.9373 - val_loss: 0.3428 - val_accuracy: 0.8624
Epoch 7/10
60/60 [===================

In [ ]:
#print(tf.__version__)

In [ ]:
import ktrain
from ktrain import text
# df = pd.read_pickle('/content/drive/MyDrive/preprodf_Augm.pkl')
# df_valid = df.sample(frac = 0.2, random_state = 42)
train_sizes = [50,70,80,100]
df_train = df.drop(df_valid.index)
for size in train_sizes:
#   df_train = df.drop(df_valid.index)
    df_tra = df_train.sample(n=size)
    (train, val, preproc) = text.texts_from_df(train_df=df_tra, 
                                               text_column='AbstractPlusTitle', 
                                               label_columns='AbL',
                                               val_df = df_valid,
                                               maxlen = 512, #length of each doc aligned to 512
                                               preprocess_mode = 'distilbert')
    bertmodel = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)
    learner = ktrain.get_learner(model = bertmodel,
                             train_data = train,
                             val_data = val,
                             batch_size = 5)
    learner.fit_onecycle(lr = 2e-5, epochs=10)
    predictor = ktrain.get_predictor(learner.model, preproc)
    textval=df_valid['AbstractPlusTitle']
    data=list(textval)
    y_pred = predictor.predict(data, return_proba=True)
    df_valid.AbL = df_valid.AbL.astype('int')
    df_tra.AbL = df_tra.AbL.astype('int')
    y_test = df_valid['AbL']
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:,1])
    auc = metrics.auc(fpr, tpr)
    print(df_tra.AbL.value_counts()) 
    print(auc)

['not_AbL', 'AbL']
     not_AbL  AbL
315      1.0  0.0
200      1.0  0.0
508      1.0  0.0
541      1.0  0.0
135      1.0  0.0
['not_AbL', 'AbL']
     not_AbL  AbL
176      1.0  0.0
77       1.0  0.0
361      1.0  0.0
90       1.0  0.0
368      1.0  0.0
preprocessing train...
language: en
train sequence lengths:
	mean : 181
	95percentile : 268
	99percentile : 355


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
10/10 [==============================] - 14s 703ms/step - loss: 0.6939 - accuracy: 0.6325 - val_loss: 0.6816 - val_accuracy: 0.6055
Epoch 2/10
10/10 [==============================] - 5s 553ms/step - loss: 0.6863 - accuracy: 0.5566 - val_loss: 0.6874 - val_accuracy: 0.5963
Epoch 3/10
10/10 [==============================] - 5s 560ms/step - loss: 0.6890 - accuracy: 0.5191 - val_loss: 0.7041 - val_accuracy: 0.3945
Epoch 4/10
10/10 [==============================] - 5s 568ms/step - loss: 0.6895 - accuracy: 0.5787 - val_loss: 0.6778 - val_accuracy: 0.6239
Epoch 5/10
10/10 [==============================] - 5s 576ms/step - loss: 0.6702 - accuracy: 0.6669 - val_loss: 0.6705 - val_accuracy: 0.6789
Epoch 6/10
10/10 [==============================] - 6s 583ms/step - loss: 0.6342 - accuracy: 0.7537 - val_loss: 0.6328 - val_accuracy: 0.6055
Epoch 7/10
10/10 [========================

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
14/14 [==============================] - 16s 596ms/step - loss: 0.6988 - accuracy: 0.3728 - val_loss: 0.6859 - val_accuracy: 0.6147
Epoch 2/10
14/14 [==============================] - 7s 490ms/step - loss: 0.6823 - accuracy: 0.5255 - val_loss: 0.6666 - val_accuracy: 0.6055
Epoch 3/10
14/14 [==============================] - 7s 497ms/step - loss: 0.6514 - accuracy: 0.5943 - val_loss: 0.6487 - val_accuracy: 0.6055
Epoch 4/10
14/14 [==============================] - 7s 505ms/step - loss: 0.6633 - accuracy: 0.5622 - val_loss: 0.6200 - val_accuracy: 0.6055
Epoch 5/10
14/14 [==============================] - 7s 513ms/step - loss: 0.5912 - accuracy: 0.6961 - val_loss: 0.5412 - val_accuracy: 0.7431
Epoch 6/10
14/14 [==============================] - 7s 514ms/step - loss: 0.4145 - accuracy: 0.8613 - val_loss: 0.4801 - val_accuracy: 0.7798
Epoch 7/10
14/14 [========================

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
16/16 [==============================] - 17s 550ms/step - loss: 0.7006 - accuracy: 0.4158 - val_loss: 0.7021 - val_accuracy: 0.3945
Epoch 2/10
16/16 [==============================] - 7s 464ms/step - loss: 0.6983 - accuracy: 0.4313 - val_loss: 0.6978 - val_accuracy: 0.3945
Epoch 3/10
16/16 [==============================] - 7s 473ms/step - loss: 0.6808 - accuracy: 0.5277 - val_loss: 0.7086 - val_accuracy: 0.3945
Epoch 4/10
16/16 [==============================] - 7s 478ms/step - loss: 0.6857 - accuracy: 0.5087 - val_loss: 0.6815 - val_accuracy: 0.4312
Epoch 5/10
16/16 [==============================] - 8s 484ms/step - loss: 0.5681 - accuracy: 0.8118 - val_loss: 0.4853 - val_accuracy: 0.8257
Epoch 6/10
16/16 [==============================] - 8s 483ms/step - loss: 0.3873 - accuracy: 0.8720 - val_loss: 0.4408 - val_accuracy: 0.8165
Epoch 7/10
16/16 [========================

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


Is Multi-Label? False
maxlen is 512
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
20/20 [==============================] - 18s 503ms/step - loss: 0.6966 - accuracy: 0.4704 - val_loss: 0.6880 - val_accuracy: 0.6147
Epoch 2/10
20/20 [==============================] - 9s 434ms/step - loss: 0.6752 - accuracy: 0.7315 - val_loss: 0.6641 - val_accuracy: 0.6055
Epoch 3/10
20/20 [==============================] - 9s 445ms/step - loss: 0.6643 - accuracy: 0.5987 - val_loss: 0.6383 - val_accuracy: 0.6422
Epoch 4/10
20/20 [==============================] - 9s 452ms/step - loss: 0.6298 - accuracy: 0.7407 - val_loss: 0.5995 - val_accuracy: 0.7982
Epoch 5/10
20/20 [==============================] - 9s 454ms/step - loss: 0.5323 - accuracy: 0.8637 - val_loss: 0.4727 - val_accuracy: 0.7890
Epoch 6/10
20/20 [==============================] - 9s 445ms/step - loss: 0.3219 - accuracy: 0.8870 - val_loss: 0.3987 - val_accuracy: 0.8257
Epoch 7/10
20/20 [========================

In [ ]:
  print(df_tra.AbL.value_counts())

0    60
1    40
Name: AbL, dtype: int64


In [ ]:
df_train = df_train.sample(n=10)

In [ ]:
df_train 

,AbstractPlusTitle,AbL
31,primary dysmenorrhea prevalent condition causi...,0
122,examine mindfulness meditation mm was associat...,0
290,haematological malignancies malignant neoplasm...,0
267,patients progressive kidney disease experience...,1
251,research impact mind body training ask partici...,0
417,cognitive behavioral therapy insomnia cbt show...,0
299,verify clinical efficacy acupuncture moxibusti...,1
511,tinnitus perception sound absence acoustic sti...,0
521,studies shown aromatherapy improve health prob...,1
26,anxiety associated sleep disturbance insomnia ...,0


In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, text_column='AbstractPlusTitle', label_columns='AbL',
                                           val_df = df_valid,
                                           maxlen = 512, #length of each doc aligned to 512
                                           preprocess_mode = 'distilbert')

['not_AbL', 'AbL']
     not_AbL  AbL
521      0.0  1.0
417      1.0  0.0
122      1.0  0.0
251      1.0  0.0
511      1.0  0.0
['not_AbL', 'AbL']
     not_AbL  AbL
176      1.0  0.0
77       1.0  0.0
361      1.0  0.0
90       1.0  0.0
368      1.0  0.0
preprocessing train...
language: en
train sequence lengths:
	mean : 195
	95percentile : 306
	99percentile : 367


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 194
	95percentile : 302
	99percentile : 550


#On 87 QA fulltext

In [ ]:
dfulQA = pd.read_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/FulltextQA Concat/dfmerge87.pkl')

In [ ]:
#dfulQA.to_excel('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/FulltextQA Concat/dfmerge87.xlsx')

In [ ]:
#dfulQA['abs_titl_qa'] = dfulQA['AbstractPlusTitle_x']+dfulQA['TreatmentGroupAnswer']+dfulQA['AnxietyMeasureGroupAnswer']+dfulQA['DepressionMeasureGroupAnswer']+dfulQA['SleepMeasureGroupAnswer']

In [ ]:
dfulQA['abs_titl_qa'] = dfulQA['AbstractPlusTitle_x']+dfulQA['TreatmentGroupAnswer']+dfulQA['SleepMeasureGroupAnswer']

In [ ]:
#dfulQA['qas'] = dfulQA['TreatmentGroupAnswer']+dfulQA['AnxietyMeasureGroupAnswer']+dfulQA['DepressionMeasureGroupAnswer']+dfulQA['SleepMeasureGroupAnswer']

In [ ]:
#dfulQA.head()

In [ ]:
def preprocess_text(dfulQA):
    dfulQA.qas = dfulQA.qas.fillna(' ')
    dfulQA.qas = dfulQA.qas.str.replace('\n',' ')
    dfulQA.qas = dfulQA.qas.str.replace('\r',' ')
    return dfulQA

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
nlp = spacy.load('en')

In [ ]:
#nlp.Defaults.stop_words -= {"will","had","was","were","did"}

In [ ]:
nlp.Defaults.stop_words |= {"None","vs."}

In [ ]:
#nlp.vocab['None'].is_stop = True

In [ ]:
new_stopwords = spacy_stopwords

In [ ]:
def clean_text(x):
  #x=re.sub(r"\'ll"," will ",x) ----  In academic paper, people tend not to use contraction
  #x=re.sub('[^a-zA-Z0-9]', ' ',x).lower()
  x = re.sub('[^a-zA-Z]', ' ',x).lower()
  x = x.split()
  new_list = []
  for word in x:
    if word not in new_stopwords: 
      new_list.append(word)
  return " ".join(new_list)

In [ ]:
dfulQA['abs_titl_qa'] = dfulQA.abs_titl_qa.apply(lambda x:clean_text(x))

In [ ]:
dfulQA

,Title_x,JournalInfo_x,Abstract_x,TiL_x,AbL_x,AbstractPlusTitle_x,PMID_x,PMCID,TreatmentGroupAnswer,SleepMeasureGroupAnswer,DepressionMeasureGroupAnswer,AnxietyMeasureGroupAnswer,InterviewAnswer,FuL,abs_titl_qa
0,"""Tai Chi Chih Compared With Cognitive Behavior...",J Clin Oncol,Purpose Cognitive behavioral therapy for insom...,1,1,Purpose Cognitive behavioral therapy for insom...,28489508,PMC5549450,tcc; cbt-i tcc; tcc v cbt-i,$ 5 points; insomnia severity index; pittsburg...,None; ids-c; ids-c,pre-sleep arousal scale; pre-sleep arousal; se...,months; 6 months 15 months; 3 months 1 month,1.0,purpose cognitive behavioral therapy insomnia ...
1,"""Cognitive behavioral therapy vs. Tai Chi for ...",Sleep,To investigate the comparative efficacy of cog...,1,1,To investigate the comparative efficacy of cog...,25142571,PMC4153053,None; comparable regard background char- acter...,athens insomnia scale; epworth sleepi- ness; a...,ids-c; clinician rating scale; ids-c,athens insomnia scale; athens insomnia; insomn...,year; None; 16 months,1.0,investigate comparative efficacy cognitive beh...
2,"""Tai chi/yoga reduces prenatal depression, anx...",Complement Ther Clin Pract,Ninety-two prenatally depressed pregnant women...,1,1,Ninety-two prenatally depressed pregnant women...,23337557,PMC3730281,tai chi/yoga; tai chi/yoga; costly therapy,None; 15-item; 15,ces-d; ces-d; frequency current depressive sym...,anxiety stai; anxiety stai scale; stai,15 min; 3 weeks gestation; 15 min,1.0,ninety prenatally depressed pregnant women ran...
3,"""Randomized controlled trial of Qigong/Tai Chi...",Ann Behav Med,Many breast cancer survivors experience fatigu...,1,1,Many breast cancer survivors experience fatigu...,25124456,PMC4329282,qg/tce versus sqg; qg/tce versus sqg; None,pittsburgh sleep quality index; None; 60–65%,beck depression inventory pittsburgh sleep qua...,depression anxiety; depression; depression anx...,short long term; 3 month follow-up; 3 months,1.0,breast cancer survivors experience fatigue moo...
4,"""Mindfulness meditation and improvement in sle...",JAMA Intern Med,Sleep disturbances are most prevalent among ol...,1,1,Sleep disturbances are most prevalent among ol...,25686304,PMC4407465,None; originally randomized conditions; nf-κb,athens insomnia scale; athens insomnia scale; ...,patient health questionnaire 9 score >14; None...,beckdepression inventory ii; beckdepression in...,10 days; 10- week; 6-week,0.0,sleep disturbances prevalent older adults untr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Sound therapy (using amplification devices and...,Cochrane Database Syst Rev,Tinnitus affects 10% to 15% of the adult popul...,1,1,Tinnitus affects 10% to 15% of the adult popul...,30589445,PMC6517157,None; combine groups; combination device (n = 15,None; 3; graded 3,hospital anxiety depression scale; hospital an...,depression; depression; tinnitus discomfort,6 months; 6 weeks; 6 weeks,0.0,tinnitus affects adult population experiencing...
83,"Efficacy of ""Pinggan Formula"" in Controlling A...",Evid Based Complement Alternat Med,To explore a new treatment that can proceed fr...,1,1,To explore a new treatment that can proceed fr...,31827555,PMC6885174,control group; control group; randomly :1 ratio,0.95 (0.40 1.50; None; 5.90± 1.40,None; 6.80± .30; 4.04 ( .95 5.1 ),None; 9.60± 1.30 9.00± .10; − 0.65 (− 1.50 0. 1,1 year; 11.5 days; 1 year,0.0,explore new treatment proceed control blood pr...
84,A Multimodal Stress-Prevention Program Supplem...,Int J Environ Res Public Health,This study compared the effectiveness of a 12-...,1,1,This study compared the effectiveness of a 12-...,33321849,PMC7763316,None; cancer; tc vs. notc,insomnia severity index; None; insomnia severi...,two-item questionnaire; self-employed’s work-r...,gad- m; generalized anxiety disorder; self-emp...,9 months; 3-week; 1 -month,0.0,study compared effectiveness day stress preven...
85,A parallel randomized controlled trial examini...,PLoS One,"This doub

In [ ]:
#dfulQA.to_excel('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/FulltextQA Concat/preproQA.xlsx')

In [ ]:
dfulQA.to_pickle('/content/drive/MyDrive/HS651_CAMs_systematic_review_automation/FulltextQA Concat/preproQA.pkl')

In [ ]:
dful = dfulQA[['abs_titl_qa','FuL']]

In [ ]:
df_valid = dful.sample(frac = 0.2, random_state = 42)

In [ ]:
df_train = dful.drop(df_valid.index)

In [ ]:
len(df_train)

70

In [ ]:
!pip3 install --upgrade tensorflow

     |████████████████████████████████| 454.3MB 36kB/s 
     |████████████████████████████████| 471kB 37.1MB/s 
     |████████████████████████████████| 4.0MB 42.7MB/s 
     |████████████████████████████████| 6.0MB 31.0MB/s 
     |████████████████████████████████| 4.0MB 47.4MB/s 
     |████████████████████████████████| 1.2MB 43.1MB/s 
     |████████████████████████████████| 4.9MB 42.4MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
    

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 114kB/s 
     |████████████████████████████████| 6.8MB 42.0MB/s 
     |████████████████████████████████| 983kB 44.8MB/s 
     |████████████████████████████████| 266kB 49.3MB/s 
     |████████████████████████████████| 1.9MB 32.3MB/s 
     |████████████████████████████████| 1.2MB 38.9MB/s 
     |████████████████████████████████| 471kB 42.9MB/s 
     |████████████████████████████████| 3.3MB 39.3MB/s 
     |████████████████████████████████| 901kB 30.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=ddc17ab30da38a1cbd41e2f1c9b564d2d28d32406e679561318264700fbf4137
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=cd4ee85c20b611066f6d8045bea3d92f85ad732d255a115c042386256258243a
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [ ]:
import tensorflow as tf
from sklearn import metrics
import ktrain
from ktrain import text

In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, 
                                               text_column='abs_titl_qa', 
                                               label_columns='FuL',
                                               val_df = df_valid,
                                               maxlen = 512, #length of each doc aligned to 512
                                               preprocess_mode = 'distilbert')

/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:605: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn('class_names implies classification but targets array contains float(s) instead of integers or strings')
/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:605: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn('class_names implies classification but targets array contains float(s) instead of integers or strings')


['not_FuL', 'FuL']
   not_FuL  FuL
1      0.0  1.0
2      0.0  1.0
3      0.0  1.0
5      1.0  0.0
6      1.0  0.0
['not_FuL', 'FuL']
    not_FuL  FuL
76      0.0  1.0
0       0.0  1.0
26      1.0  0.0
22      0.0  1.0
12      0.0  1.0



preprocessing train...
language: en
train sequence lengths:
	mean : 186
	95percentile : 247
	99percentile : 357


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 191
	95percentile : 239
	99percentile : 248


In [ ]:
bertmodel = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 512
done.


In [ ]:
learner = ktrain.get_learner(model = bertmodel,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=10)

KeyboardInterrupt: ignored

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
#predictor.save('/content/drive/MyDrive/DistillBert_TitlePlusAbstract')

In [ ]:
texts=df_valid['abs_titl_qa']
data=list(texts)

In [ ]:
#data

In [ ]:
###predictor.get_classes()

In [ ]:
y_pred = predictor.predict(data, return_proba=True)

In [ ]:
df_valid.FuL = df_valid.FuL.astype('int')
df_train.FuL = df_train.FuL.astype('int')

In [ ]:
y_test = df_valid['FuL']

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred[:,1])
auc = metrics.auc(fpr, tpr)
#plt.plot(fpr, tpr)
auc

In [ ]:
#### Alternate metric reports ####

In [ ]:
X_test = df_valid.loc [:,'last500']
X_test[78]

In [ ]:
X_test = df_valid.loc [:,'last500']

# X_test.keys()
X_test_BERT = []
for keys in X_test.keys():
    X_test_BERT.append(X_test[keys])
#X_test_BERT[0]

In [ ]:
y_pred_BERT = predictor.predict(X_test_BERT, return_proba=True)

In [ ]:
#predictor.save('/content/drive/MyDrive/BERT_HS651CAMs_pred')

In [ ]:
y_test = df_valid['AbL']

In [ ]:
y_pred_BERT

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_BERT[:,1])
auc = metrics.auc(fpr, tpr)
auc

In [ ]:
# Load saved model

In [ ]:
saved_predictor = ktrain.load_predictor('/content/drive/MyDrive/BERT_HS651CAMs_pred')

In [ ]:
y_pred_BERT1 = saved_predictor.predict(X_test_BERT, return_proba=True)

In [ ]:
y_pred_BERT1 

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_BERT1[:,1])
auc = metrics.auc(fpr, tpr)
auc